In [1]:
conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-surprise-1.1.1      |   py38h1e00858_1         567 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         567 KB

The following NEW packages will be INSTALLED:

  scikit-surprise    conda-forge/win-64::scikit-surprise-1.1.1-py38h1e00858_1




scikit-surprise-1.1. | 567 KB    |            |   0% 
scikit-surprise-1.1. | 567 KB    | 2          |   3% 
scikit-surprise-1.1. | 567 KB    | ###9       |  40% 
scikit-surprise-1.1. | 567 KB    | ########## | 100% 
scikit-surprise-1.1. | 567 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Exec

In [3]:
import surprise
print(surprise.__version__)

1.1.1


## Surprise를 이용한 추천 시스템 구축

In [3]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [5]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\최현정/.surprise_data/ml-100k


In [6]:
algo = SVD()
algo.fit(trainset)

In [7]:
predictions = algo.test(testset)
print('prediction type :', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.342854417495433, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.622257219702484, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.211577684174389, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.2588473620334897, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.2625905477894084, details={'was_impossible': False})]

In [8]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.342854417495433),
 ('882', '291', 3.622257219702484),
 ('535', '507', 4.211577684174389)]

In [9]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.07   {'was_impossible': False}


In [10]:
accuracy.rmse(predictions)

RMSE: 0.9495


0.9495480819362355

## Surprise 주요 모듈 소개

In [1]:
import pandas as pd

ratings = pd.read_csv('D:/머신러닝 완벽가이드 데이터/ml-latest-small/ratings.csv')
ratings.to_csv('D:/머신러닝 완벽가이드 데이터/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [4]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('D:/머신러닝 완벽가이드 데이터/ml-latest-small/ratings_noh.csv', reader = reader)

In [5]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [6]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('D:/머신러닝 완벽가이드 데이터/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## 교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝

In [7]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('D:/머신러닝 완벽가이드 데이터/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv = 5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8725  0.8740  0.8710  0.8760  0.8733  0.8733  0.0016  
MAE (testset)     0.6715  0.6734  0.6685  0.6724  0.6680  0.6708  0.0021  
Fit time          4.09    4.10    4.11    4.11    4.08    4.10    0.01    
Test time         0.10    0.10    0.11    0.51    0.11    0.19    0.16    


{'test_rmse': array([0.87249713, 0.87395454, 0.87102268, 0.8759843 , 0.87328173]),
 'test_mae': array([0.67151197, 0.67340928, 0.6685418 , 0.67236068, 0.66801699]),
 'fit_time': (4.09206485748291,
  4.102588415145874,
  4.111531972885132,
  4.1137237548828125,
  4.083117485046387),
 'test_time': (0.10272479057312012,
  0.09873580932617188,
  0.10870885848999023,
  0.5116140842437744,
  0.10773491859436035)}

In [8]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv = 3)
# 위의 ratings.csv 를 DataFrame으로 로딩한 데이터
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8778627435911618
{'n_epochs': 20, 'n_factors': 50}


## Surprise 를 이용한 개인화 영화 추천 시스템 구축

In [10]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
# ERROR
#algo.fit(data)

In [12]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5.0))
data_folds = DatasetAutoFolds(ratings_file='D:/머신러닝 완벽가이드 데이터/ml-latest-small/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [13]:
algo = SVD(n_epochs=20, n_factors = 50, random_state=0)
algo.fit(trainset)

In [16]:
movies = pd.read_csv('D:/머신러닝 완벽가이드 데이터/ml-latest-small/movies.csv')

# userId = 9 의 movieId 데이터 추출하여 movieId = 42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds == 42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [17]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [18]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:', len(seen_movies), '추천대상 영화수:', len(unseen_movies), 
         '전체 영화수:', len(total_movies))
    return unseen_movies
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [19]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 10):
    
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est

    predictions.sort(key = sortkey_est, reverse = True)
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    
    # (movies.movieId)에 (top_movie_ids)가 포함돼 있는 영화 제목 추출
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    # 영화제목, 영화평점 출력
    print(top_movie[1], ':', top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
